In [1]:
import numpy as np
import trainning_of_adaboost as toa
from sklearn.ensemble import AdaBoostClassifier
import ImAda_DecisionTree
from sklearn.metrics import accuracy_score, confusion_matrix, roc_auc_score, f1_score, precision_score
import math
from sklearn.ensemble import AdaBoostClassifier
from datetime import datetime
import csv
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import StandardScaler
from wsvm.application import Wsvm
from sklearn.svm import SVC

In [2]:
def compute_metrics(y_test,y_pred):
    cm_WSVM = confusion_matrix(y_test, y_pred)
    se = cm_WSVM[1,1]/(cm_WSVM[1,0]+cm_WSVM[1,1])
    sp = cm_WSVM[0,0]/(cm_WSVM[0,0]+cm_WSVM[0,1])
    gmean = math.sqrt(se*sp)
    f1s = f1_score(y_test,y_pred)
    acc = accuracy_score(y_test,y_pred)
    auc = roc_auc_score(y_test, y_pred)

    return sp, se, gmean, f1s, acc, auc

In [3]:
# 4. WSVM
def wsvm(C,X_train, y_train,X_test,distribution_weight=None):
    model = Wsvm(C,distribution_weight)
    model.fit(X_train, y_train)
    test_pred = model.predict(X_test)
    return test_pred

In [4]:
# 6. AdaBoost DecisionTree
def ada_decisiontree(M, X_train, y_train,X_test):
    clf = AdaBoostClassifier(n_estimators=M)
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    return y_pred

In [5]:
# 7. AdaBoost WSVM
def ada_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization=True, proposed_preprocessing = False,proposed_alpha = False, test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [6]:
# 8. IM.AdaBoost-1 WSVM
def imada1_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization=True, proposed_preprocessing = True, proposed_alpha = False, test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [7]:
#9. IM.AdaBoost-2 WSVM
def imada2_wsvm(M, C, X_train, y_train,X_test):
    w, b, a= toa.fit(X_train, y_train, M, C, instance_categorization = True,proposed_preprocessing = False,proposed_alpha=True,test_something = False)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [8]:
#10. IM.AdaBoost-12 WSVM
def imada_12_wsvm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization = True,proposed_preprocessing= True,proposed_alpha=True,test_something = False, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [9]:
#11. IM.AdaBoost1 + SVM
def imada1_svm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization = False, proposed_preprocessing= True,proposed_alpha=False,test_something = True, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred


In [10]:
#12. IM.AdaBoost2 + SVM
def imada2_svm(M, C, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization = False,proposed_preprocessing = False,proposed_alpha=True,test_something = True)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [11]:
#13. IM.AdaBoost12 + SVM
def imada_12_svm(M, C, theta, X_train, y_train,X_test):
    w, b, a = toa.fit(X_train, y_train, M, C, instance_categorization = False,proposed_preprocessing= True,proposed_alpha=True,test_something = True, theta=theta)
    y_pred = toa.predict(X_test, w, b, a, M)
    return y_pred

In [12]:
#14. IM.AdaBoost1 + DS
def imada_1_decisiontree(M,theta,X_train, y_train,X_test):
    clf, a = ImAda_DecisionTree.fit(X_train, y_train, M, proposed_preprocessing = True, proposed_alpha = False, theta = theta)
    y_pred = ImAda_DecisionTree.predict(X_test, a, clf)
    return y_pred

In [13]:
#15. IM.AdaBoost2 + DS
def imada_2_decisiontree(M, X_train, y_train,X_test):
    clf, a = ImAda_DecisionTree.fit(X_train, y_train, M, proposed_preprocessing = False, proposed_alpha = True)
    y_pred = ImAda_DecisionTree.predict(X_test, a, clf)
    return y_pred

In [14]:
#16. IM.AdaBoost2 + DS
def imada_12_decisiontree(M,theta,X_train, y_train,X_test):
    clf, a = ImAda_DecisionTree.fit(X_train, y_train, M, proposed_preprocessing = True, proposed_alpha = True, theta = theta)
    y_pred = ImAda_DecisionTree.predict(X_test, a, clf)
    return y_pred

In [15]:
################################# K-FOLD SCRIPT FOR CO-AUTHOR DATASETS-MAIN ##############################
N = 10
M = [10]
C = [10000]
theta = [0.2, 0.5, 0.8, 1.1, 1.4, 1.7, 2]

from ProcessingData import CoAuthor_1, CoAuthor_2, CoAuthor_3, CoAuthor_4, CoAuthor_5, CoAuthor_6, CoAuthor_7, CoAuthor_8, CoAuthor_9, CoAuthor_10
dataset = CoAuthor_1
beta_center, beta_estimate, beta_actual = 0.6, 1, 0.3

time = datetime.now().strftime("%d%m%Y_%H%M%S")
filename = (str(dataset).split("\\")[-1]).split(".")[0]


for n in range(0,N):
    header = ['Times','Fold','Name Method', 'M','C','theta','SP', 'SE', 'Gmean', 'F1 Score','Precision','Accuracy','AUC']
    X, y = dataset.load_data()
    print(X.shape)
    kfold_validation = StratifiedKFold(n_splits=5, shuffle=True) 
    with open(f'./Experiment/{filename}_{time}.csv', 'a', encoding='UTF8', newline='') as f3:
        writer = csv.writer(f3)
        writer.writerow(header)
        fold = 1
        for train_index, test_index in kfold_validation.split(X,y):
            print("\nN = ",n+1,", Fold = ",fold)
            # X_train, y_train = X.iloc[train_index], y.iloc[train_index]
            # X_test, y_test = X.iloc[test_index], y.iloc[test_index]
            X_train, y_train = X[train_index], y[train_index]
            X_test, y_test = X[test_index], y[test_index]
            # print(X_test.shape)
            
            #Scalling Data
            sc_X = StandardScaler()
            X_train = sc_X.fit_transform(X_train)
            X_test = sc_X.transform(X_test)
            y_train = np.array(y_train)

            ############ USING IM.AdaBoost Method #####################
            for m in M:
                print("M = ", m)
                #No 9
                print("Im.AdaBoost2.DecisionTree starting...\n")
                y_pred = imada_2_decisiontree(m, X_train, y_train,X_test)
                sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                name = 'Im.AdaBoost.DecisionTree.2'
                c = "None"
                t = "None"
                writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])
                    

                for t in theta:
                    print("theta = ",t)
                    #No 8
                    print("ImADA_1_DecisionTree starting...\n")
                    c = "None"
                    y_pred = imada_1_decisiontree(m, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                    name = 'Im.AdaBoost.DecisionTree.1'
                    writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])

                    #No 10
                    print("ImADA_12_DecisionTree starting...\n")
                    y_pred = imada_12_decisiontree(m, t, X_train, y_train,X_test)
                    sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                    c = "None"
                    name = 'Im.AdaBoost.DecisionTree.12'
                    writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])

            for m in M:
                for c in C:
                    for t in theta:
                        print(m,c,t)
                        #No 11
                        print("ImADA1_SVM starting...\n")
                        y_pred = imada1_svm(m, c, t, X_train, y_train,X_test)
                        sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                        name = 'Im.ADABoost.SVM.1'
                        writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])

                        #No 12
                        print("ImADA2_SVM starting...\n")
                        y_pred = imada2_svm(m, c, X_train, y_train,X_test)
                        sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                        name = 'Im.ADABoost.SVM.2'
                        writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])

                        #No 13
                        print("ImADA_12_SVM starting...\n")
                        y_pred = imada_12_svm(m, c,t, X_train, y_train,X_test)
                        sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                        name = 'Im.ADABoost.SVM.12'
                        writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])

                        #No 14
                        print("ImADA1_WSVM starting...\n")
                        y_pred = imada1_wsvm(m, c, t, X_train, y_train,X_test)
                        sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                        name = 'Im.ADABoost.W-SVM.1'
                        writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])

                        #No 15
                        print("ImADA2_WSVM starting...\n")
                        y_pred = imada2_wsvm(m, c, X_train, y_train,X_test)
                        sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                        name = 'Im.ADABoost.W-SVM.2'
                        writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])   

                        #No 16
                        print("ImADA_12_WSVM starting...\n")
                        y_pred = imada_12_wsvm(m, c, t, X_train, y_train,X_test)
                        sp, se, gmean, f1s, acc, auc = compute_metrics(y_test, y_pred)
                        name = 'Im.ADABoost.W-SVM.12'
                        writer.writerow([n+1,fold,name,m,c,t,sp, se, gmean, f1s, acc, auc])   
            
            fold = fold + 1

(400, 7)

N =  1 , Fold =  1
M =  10
Im.AdaBoost2.DecisionTree starting...

theta =  0.2
ImADA_1_DecisionTree starting...

ImADA_12_DecisionTree starting...

theta =  0.5
ImADA_1_DecisionTree starting...

ImADA_12_DecisionTree starting...

theta =  0.8
ImADA_1_DecisionTree starting...

ImADA_12_DecisionTree starting...

theta =  1.1
ImADA_1_DecisionTree starting...

ImADA_12_DecisionTree starting...

theta =  1.4
ImADA_1_DecisionTree starting...

ImADA_12_DecisionTree starting...

theta =  1.7
ImADA_1_DecisionTree starting...



ImADA_12_DecisionTree starting...

theta =  2
ImADA_1_DecisionTree starting...

ImADA_12_DecisionTree starting...

10 10000 0.2
ImADA1_SVM starting...

ImADA2_SVM starting...

ImADA_12_SVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...

10 10000 0.5
ImADA1_SVM starting...

ImADA2_SVM starting...

ImADA_12_SVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...

10 10000 0.8
ImADA1_SVM starting...

ImADA2_SVM starting...

ImADA_12_SVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...

10 10000 1.1
ImADA1_SVM starting...

ImADA2_SVM starting...

ImADA_12_SVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...

10 10000 1.4
ImADA1_SVM starting...

ImADA2_SVM starting...

ImADA_12_SVM starting...

ImADA1_WSVM starting...

ImADA2_WSVM starting...

ImADA_12_WSVM starting...

10 10000 1.7
ImADA1_SVM starting...

ImADA2_SVM starting...

ImAD